In [1]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
from sagemaker.session import Session
from time import time

import pandas as pd

In [2]:
# ロールを取得する
role = get_execution_role()
print(role)

arn:aws:iam::326497581172:role/service-role/AmazonSageMaker-ExecutionRole-20230418T103673


In [15]:
#使用するバケットを指定してください
bucket="trolley-monitor"
session = Session(default_bucket=bucket)
app_name = "sample_data"
workflow_id = str(time()).replace(".", "")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [16]:
input_data_uri = f's3://{bucket}/{app_name}/adult/adult.data'

In [17]:
INPUT_DESTINATION = "/opt/ml/processing/input"
OUTPUT_SOURCE = "/opt/ml/processing/output"

In [18]:
sklearn_processor = SKLearnProcessor(
    framework_version='1.0-1',
    role=role,
    instance_type='ml.t3.medium',
    instance_count=1,
    sagemaker_session=session
)

sklearn_processor.run(
    code="profiling.py",
    inputs=[
        ProcessingInput(
            source=input_data_uri,
            destination=INPUT_DESTINATION
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="profiling",
            source=OUTPUT_SOURCE,
            destination=f"s3://{bucket}/{app_name}/{workflow_id}/profiling"
        ),
    ],
    arguments=[
        "--input_file", input_data_uri,
        "--output_dir", OUTPUT_SOURCE,
        "--output_file", "profiling.html"
    ],
    wait=False,
    job_name="trolley-monitor-profiling" + workflow_id
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker:Creating processing-job with name trolley-monitor-profiling16902702543032303
